## Auntoencoder

[参考]
https://blog.keras.io/building-autoencoders-in-keras.html

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

### MNISTデータの読み込み

In [ ]:
# Load the MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../DAY7/2_notebook/mnist_data/", one_hot=True) #パスは適宜変更する
X_train = mnist.train.images
X_test = mnist.test.images
train_labels = mnist.train.labels
test_labels = mnist.test.labels

### モデルの構築

In [ ]:
encoding_dim = 36

# 入力層の定義
input_img = Input(shape=(784,))
# 入力層〜中間層を定義
encoded = Dense(encoding_dim, activation='relu')(input_img)
# 中間層〜出力層を定義
decoded = Dense(784, activation='sigmoid')(encoded)
# 入力層〜出力層までをつなげて、ネットワークを完成させる
autoencoder = Model(input=input_img, output=decoded)
# 計算条件の定義
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# 計算結果を利用しやすくするために、以下のモデルを定義
# エンコーダ部分だけのモデルを定義
encoder = Model(input_img, encoded)
# デコーダ部分だけのモデルを定義
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

### オートエンコーダの実行

In [ ]:
autoencoder.fit(X_train, X_train,
                nb_epoch=10,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

### テストデータを入力し、中間層と出力層の値を得る

In [ ]:
encoded_imgs = encoder.predict(X_test)
decoded_imgs = decoder.predict(encoded_imgs)
# decoded_imgs = autoencoder.predict(X_test) #としても同じ

### 結果の可視化

In [ ]:
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # 中間層の値
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(6,6)) #画像サイズは、encoding_dimに合わせて変更する
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display reconstruction
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### [演習]
* Epoch数を減らすと再構成画像がどうなるか確認してみましょう
* 中間層のノード数を減らすと再構成画像がどうなるか確認してみましょう